In [1]:
import pandas as pd
import functools
import numpy as np

In [31]:
class Contacts:
    def __init__(self):
        self.Contacts = pd.read_csv("../input/google contacts.csv")
        self.fixNames()
        self.b = len(self.Contacts)
        self.Contacts = self.Contacts.drop_duplicates(ignore_index=True)
        self.dup = self.b - len(self.Contacts) 
        self.add()
        self.a = len(self.Contacts)
        self.Contacts["Given Name"] = self.Contacts["Name"]
        self.printResults()

    def printResults(self):
        print("before:",self.b, "after:",self.a, "# of duplicates:",self.dup, "# compressed:", self.b-self.a)
        i = self.Contacts.groupby("Name")["Phone 1 - Value"].count()
        print("# of different people (unique first and last names):", len(i))
        
    def fixNames(self):
        v = self.Contacts.loc[~self.Contacts["Name"].isnull() & self.Contacts["Name"].str.contains("'"),"Name"].values.tolist()
        newNames = [name.strip("'") if name[0]=="'" else name for name in v ]
        self.Contacts.loc[~self.Contacts["Name"].isnull() & self.Contacts["Name"].str.contains("'"),"Name"] = newNames

    
    def merge(self,person):
        newperson = []
        for feature in person.columns:
            vals = list(person[feature].dropna().unique())
            if len(vals)==0:
                vals = [None]
            newF = functools.reduce(lambda a, b: str(a)+" "+str(b), vals)
            newperson.append(newF)
        return newperson
    def createNewPersonContacts(self, person):
        less_than_one_phone = len(person["Phone 1 - Value"].dropna().unique())<=1
        less_than_one_email = len(person["E-mail 1 - Value"].dropna().unique())<=1
        if less_than_one_phone and less_than_one_email:
            newperson = [self.merge(person)]
        elif not less_than_one_email and less_than_one_phone:
            newperson = person.groupby("E-mail 1 - Value").apply(self.merge).values
        elif not less_than_one_phone and less_than_one_email: 
            newperson = person.groupby("Phone 1 - Value").apply(self.merge).values
        else:
            newperson = person.groupby("E-mail 1 - Value").apply(self.merge).values
        newperson = list(newperson)
        return newperson
                
                
    def add(self):
        c = self.Contacts.groupby("Name",dropna=False)
        newC = []
        o = 0
        for i,person in c:
            newperson = self.createNewPersonContacts(person) 
            o += len(person)
            newC += newperson
        
        print(len(self.Contacts),o)
            
        self.Contacts = pd.DataFrame(newC,columns=self.Contacts.columns)
        self.Contacts = self.Contacts.sort_values(by="Name")

In [32]:
c = Contacts()
c.Contacts.to_csv("../output/newGoogleContacts.csv",index=False)

4121 4121
before: 6775 after: 3810 # of duplicates: 2654 # compressed: 2965
# of different people (unique first and last names): 3549


In [37]:
c.Contacts.iloc[0:1500].to_csv("../output/parts new gcontacts/gcontacts-p1.csv")
c.Contacts.iloc[1500:3000].to_csv("../output/parts new gcontacts/gcontacts-p2.csv")
c.Contacts.iloc[3000:len(c.Contacts)].to_csv("../output/parts new gcontacts/gcontacts-p3.csv")

In [388]:
c.Contacts.to_csv("newContacts.csv",index=False)

In [381]:
np.nan != np.nan

True

In [391]:
c.Contacts.iloc[20:24].to_csv("practice Contacts.csv",index=False)